In [ ]:
Konteks: Optimasi Formula Minuman Isotonik
Dalam pengembangan minuman isotonik, kita ingin menemukan komposisi optimal dari tiga bahan utama:
A: Konsentrasi Gula (%) → (2% - 6%)
B: Konsentrasi Garam (%) → (0.1% - 0.5%)
C: Konsentrasi Asam Sitrat (%) → (0.05% - 0.2%)

Total komposisi harus selalu berjumlah 100% (karena ini adalah mixture), sehingga kita tidak bisa memvariasikan faktor secara independen seperti pada desain eksperimen biasa.
Kita menggunakan Simplex-Lattice Design, yang umum dalam Mixture Design, untuk mengevaluasi berbagai kombinasi bahan.

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations_with_replacement

In [ ]:
# 1. Definisi level untuk Simplex-Lattice Design (q=3, m=3)
levels = [0, 1/3, 2/3, 1]  # Proporsi dalam skala 0-1
design_points = list(combinations_with_replacement(levels, 3))

In [ ]:
# 2. Filter kombinasi agar total selalu 1
design_points = [p for p in design_points if np.isclose(sum(p), 1)]

In [ ]:
# 3. Konversi proporsi ke skala aktual (%)
A_min, A_max = 2, 6  # Gula
B_min, B_max = 0.1, 0.5  # Garam
C_min, C_max = 0.05, 0.2  # Asam Sitrat

df = pd.DataFrame(design_points, columns=["A", "B", "C"])
df["A"] = df["A"] * (A_max - A_min) + A_min
df["B"] = df["B"] * (B_max - B_min) + B_min
df["C"] = df["C"] * (C_max - C_min) + C_min

In [ ]:
# 4. Simulasikan respons (Kelezatan, skala 1-10)
np.random.seed(42)
df["Kelezatan"] = (
    5 + 1.2 * df["A"] - 3.5 * df["B"] + 2.5 * df["C"]
    - 0.8 * df["A"] * df["B"] - 1.2 * df["A"] * df["C"]
    + np.random.normal(0, 0.3, len(df))
)

In [ ]:
# 5. Analisis Regresi Kuadratik
model = smf.ols('Kelezatan ~ A + B + C + A:B + A:C + B:C', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

In [ ]:
# 6. Visualisasi Data
sns.scatterplot(data=df, x="A", y="Kelezatan", hue="B", size="C", palette="coolwarm")
plt.title("Pengaruh Komposisi terhadap Kelezatan Minuman")
plt.xlabel("Konsentrasi Gula (%)")
plt.ylabel("Skor Kelezatan")
plt.grid(True)
plt.show()

In [ ]:
# 7. Output hasil
print(df)
print("\nANOVA Table:\n", anova_table)
print("\nModel Summary:\n", model.summary())